In [10]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import pickle
from utils.cleansing import *
from utils.models import *


predictions_submit = regression.predict(X_pred)
predictions_submit

submission = pd.DataFrame({'id': X_pred.index, 'price': predictions_submit})

submission.to_csv(r'.\data\processed\submission_1.csv', index=False)

submission.head()


In [11]:
# Usar diccionario de modelos al pasar a .py

In [12]:
# Para la primera ronda, aplicamos una regresión lineal a los datos sin hacer ningún tratamiento
df_train = pd.read_csv(r'.\data\processed\diamonds_train_1.csv').set_index('id')
X_pred = pd.read_csv(r'.\data\processed\diamonds_test_1.csv').set_index('id')

X = df_train.drop(columns='price')
y = df_train['price']

regression = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=43)
regression.fit(X_train, y_train)
y_pred = regression.predict(X_test)

for metric in evaluate_metrics(y_test, y_pred):
    print(metric)


rmse: 0.22253585420505206
mse: 0.04952220640677219
mae: 0.12249590644656809
r2 score: 0.9526975928771463
mape: 0.01582015504023537
